## This notebook explores 36 records that were kicked back from Lex's R code. 
### The output of this notebook is:
#### 1. 6 Lat/Long calculated for FAA Airport Identifier
#### 2. 22  Lat/Long calculated for Runways
#### 3. 2 Records (back to Lex)

In [1]:
# Import all needed libraries
import pandas as pd
import numpy as np
import re
import geopy
from geopy.distance import geodesic

In [81]:
# Read in csv file
nonstd = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/1stPass_All_Points_Non-std-479Runways.csv')
nonstd

,REMARKS,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION
0,Reported a UAS .25 NM N of their position whil...,['rwy27 '],"['25 NM N', '1 NM ']",[],1/30/2018,10:11,2018-01-30T10:11Z,NKS787,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM N of SAN,RWY 27
1,Aircraftreported a UAS 500 feet off their righ...,['rwy27r'],['18 NM NE'],"['27R', 'ORD']",2/19/2018,4:24,2018-02-19T04:24Z,ENY3709,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,18 NM NE ORD,RWY 27R
2,ATCT received report of a UAS flying .5 NM N o...,[],['5 NM N'],['TCT'],3/1/2018,7:59,2018-03-01T07:59Z,NaN,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),.5 NM N of FAT,NaN
3,VFR aircraft reported observing a UAS pass wit...,['rwy23 '],[],[],4/21/2018,2:33,2018-04-21T02:33Z,N7425T,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM S SW1,RWY 23
4,Reported responding to two TCAS RA's over an o...,['rwy09'],[],['FNT'],6/21/2018,7:20,2018-06-21T07:20Z,IRO7552,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,6 NM W of FNT,RWY 09
5,Reported a UAS on a 6 NM final to RWY 22l. UAS...,['rwy22l'],"['6 NM ', '1 NM N']",[],7/3/2018,20:21,2018-07-03T20:21Z,SWA1156,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of BOS,RWY 22
6,PIC reported a white UAS off right side at sam...,['rwy17r'],['1 NM NNW'],['00F'],7/26/2018,19:21,2018-07-26T19:21Z,UPS2805,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM NNW SDF,RWY 17R
7,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,['rwy20 '],['25 NM E'],['50F'],8/16/2018,3:44,2018-08-16T03:44Z,N93EC,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,0.25 NM E MSY,RWY 20
8,"Reported observing a white UAS at 2,500 feet, ...",['rwy13 '],[],['LGA'],8/20/2018,5:11,2018-08-20T05:11Z,SKW3610,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,2 NM S of N90,RWY 13
9,Observed a black UAS with flashing red lights ...,['rwy25l'],['5 NM SE'],[],8/28/2018,2:22,2018-08-28T02:22Z,MSQT384,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1.5 NM SE of P50,RWY 25L


In [82]:
# Find all records that reference airports and save them in new dataframe
nonstd_air = nonstd[(nonstd['DATASET'] == 'AIRPORT') | (nonstd['DATASET'] == 'AIRPORT(?)')]
nonstd_air

,REMARKS,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION
2,ATCT received report of a UAS flying .5 NM N o...,[],['5 NM N'],['TCT'],3/1/2018,7:59,2018-03-01T07:59Z,NaN,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),.5 NM N of FAT,NaN
16,N9042Q reported a UAS while on right base for ...,['rwy 15 '],[],[],11/14/2018,0:44,2018-11-14T00:44Z,N9042Q,CONR,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),0.5 NM S of BUR,NaN
18,Received a report via telephone of a UAS sight...,['rwy 1r'],[],"['IAD', 'CRC', 'IAD', 'TCT', 'IAD']",1/29/2019,4:35,2019-01-29T04:35Z,NaN,NCRCC,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),2 NM S of NCRCC,NaN
19,UAS:Fairfax County Police reported sighting a ...,['rwy 1r'],[],"['IAD', 'DCA', 'CRC', 'IAD', 'TCT']",2/4/2019,22:05,2019-02-04T22:05Z,NaN,NCRCC,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),1 NM E of NCRCC,NaN
21,Reported black and white UAS at 1200 feet whil...,['rwy 22 '],[],['LGA'],3/4/2019,2:09,2019-03-04T02:09Z,DAL1788,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT,3 nm SE LGA,NaN
27,Reported observing a UAS operating over the 10...,[],['1 NM S'],[],6/21/2019,5:00,2019-06-21T05:00Z,N741LS,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),1 NM S of EMT,NaN
28,Sacramento Airport Authority notified of a mod...,[],['25 NM N'],['AWO'],8/10/2020,17:55,2020-08-10T17:55Z,NaN,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),.25 NM N of SMF,NaN
29,DCA-2: Aircraft landing DCA runway 1 observed ...,['rwy 1 '],['5 NM S'],"['DCA', 'DCA', 'DCA', 'DCA', 'DCA', 'MOR']",9/18/2020,20:03,2020-09-18T20:03Z,SWA961,NCRCC,TOI-NCR FRZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),5 NM S of DCA,NaN


In [83]:
# Extract all IDENTs
uas_loc = nonstd_air['UASLOCATION'].tolist()
uas_airport = []

for i in range(len(uas_loc)):
    airport = uas_loc[i].split(' ')[-1]
    
    if len(airport) <= 4:
        uas_airport.append(airport)
    else:
        trim_air = airport[-3:]
        uas_airport.append(trim_air)
        
uas_airport_df = pd.DataFrame()
uas_airport_df['IDENT'] = uas_airport
uas_airport_df

,IDENT
0,FAT
1,BUR
2,RCC
3,RCC
4,LGA
5,EMT
6,SMF
7,DCA


In [5]:
# Read in the airports dataset
airports = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/airports_cleaned.csv')

In [20]:
# Merge the airports dataset with the IDENTs dataframe
uas_air_loc= pd.merge(uas_airport_df, airports, on='IDENT', how='left')
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,FAT,Fresno Yosemite Intl,36.776556,-119.718833,KFAT
1,BUR,Bob Hope,34.200694,-118.358667,KBUR
2,RCC,NaN,NaN,NaN,NaN
3,RCC,NaN,NaN,NaN,NaN
4,LGA,Laguardia,40.777250,-73.872611,KLGA
5,EMT,San Gabriel Valley,34.086009,-118.034845,KEMT
6,SMF,Sacramento Intl,38.695444,-121.590778,KSMF
7,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [21]:
# Add Remarks and UAS location columns to dataframe
add_col = nonstd_air['REMARKS'].tolist()
uas_air_loc['REMARKS'] = add_col
add_col1 = nonstd_air['UASLOCATION'].tolist()
uas_air_loc['UASLOCATION'] = add_col1
uas_air_loc

,IDENT,NAME,LATITUDE,LONGITUDE,ICAO_ID,REMARKS,UASLOCATION
0,FAT,Fresno Yosemite Intl,36.776556,-119.718833,KFAT,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT
1,BUR,Bob Hope,34.200694,-118.358667,KBUR,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR
2,RCC,NaN,NaN,NaN,NaN,Received a report via telephone of a UAS sight...,2 NM S of NCRCC
3,RCC,NaN,NaN,NaN,NaN,UAS:Fairfax County Police reported sighting a ...,1 NM E of NCRCC
4,LGA,Laguardia,40.777250,-73.872611,KLGA,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA
5,EMT,San Gabriel Valley,34.086009,-118.034845,KEMT,Reported observing a UAS operating over the 10...,1 NM S of EMT
6,SMF,Sacramento Intl,38.695444,-121.590778,KSMF,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF
7,DCA,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA


In [22]:
# Creating dictionary for bearing degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate the distances and bearings into two different columns
uas_loc = uas_air_loc['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: 
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create new dataframe for UAS locations
new_df = pd.DataFrame(uas_air_loc['REMARKS'])
new_df['UAS_LOC'] = uas_air_loc['UASLOCATION']
new_df['IDENT'] = uas_air_loc['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing
0,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT,FAT,.5,N
1,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR,BUR,0.5,S
2,Received a report via telephone of a UAS sight...,2 NM S of NCRCC,RCC,2,S
3,UAS:Fairfax County Police reported sighting a ...,1 NM E of NCRCC,RCC,1,E
4,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA,LGA,3,SE
5,Reported observing a UAS operating over the 10...,1 NM S of EMT,EMT,1,S
6,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF,SMF,.25,N
7,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA,DCA,5,S


In [61]:
# Converting distances from NM to kilometers and convert bearing abbreviations to their corresponding degrees
new_df['Distance'] = pd.to_numeric(new_df['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(new_df)):
    distanceKilo = new_df['Distance'][i] * 1.852 
    dist_kilo.append(distanceKilo)

bearDegree = pd.DataFrame(new_df['Bearing'])
bearDegree = bearDegree.replace({'Bearing': bearing_deg})

# Append all new columns to UAS location dataframe
new_df['Distance_Kilometers'] = dist_kilo
new_df['Bearing_Degrees'] = bearDegree
new_df['Airport_Latitude'] = uas_air_loc['LATITUDE']
new_df['Airport_Longitude'] = uas_air_loc['LONGITUDE']
new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT,FAT,0.50,N,0.926,0,36.776556,-119.718833
1,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR,BUR,0.50,S,0.926,180,34.200694,-118.358667
2,Received a report via telephone of a UAS sight...,2 NM S of NCRCC,RCC,2.00,S,3.704,180,NaN,NaN
3,UAS:Fairfax County Police reported sighting a ...,1 NM E of NCRCC,RCC,1.00,E,1.852,90,NaN,NaN
4,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA,LGA,3.00,SE,5.556,135,40.777250,-73.872611
5,Reported observing a UAS operating over the 10...,1 NM S of EMT,EMT,1.00,S,1.852,180,34.086009,-118.034845
6,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF,SMF,0.25,N,0.463,0,38.695444,-121.590778
7,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA,DCA,5.00,S,9.260,180,38.851444,-77.037722


In [62]:
# Drop unnecessary columns and rename needed columns
new_df2 = pd.merge(new_df, airports, on='IDENT', how='left')
new_df2 = new_df2.drop(['Airport_Latitude', 'Airport_Longitude'], 1)
new_df2 = new_df2.rename({'LONGITUDE': 'Airport_Longitude', 'LATITUDE': 'Airport_Latitude'}, axis=1)
new_df2

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT,FAT,0.50,N,0.926,0,Fresno Yosemite Intl,36.776556,-119.718833,KFAT
1,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR,BUR,0.50,S,0.926,180,Bob Hope,34.200694,-118.358667,KBUR
2,Received a report via telephone of a UAS sight...,2 NM S of NCRCC,RCC,2.00,S,3.704,180,NaN,NaN,NaN,NaN
3,UAS:Fairfax County Police reported sighting a ...,1 NM E of NCRCC,RCC,1.00,E,1.852,90,NaN,NaN,NaN,NaN
4,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA,LGA,3.00,SE,5.556,135,Laguardia,40.777250,-73.872611,KLGA
5,Reported observing a UAS operating over the 10...,1 NM S of EMT,EMT,1.00,S,1.852,180,San Gabriel Valley,34.086009,-118.034845,KEMT
6,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF,SMF,0.25,N,0.463,0,Sacramento Intl,38.695444,-121.590778,KSMF
7,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA,DCA,5.00,S,9.260,180,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [63]:
# Find unmatched columns and save them in separate dataframe for further investigation
null = new_df2[new_df2['NAME'].isnull()]
null = null.drop(['IDENT', 'NAME', 'Airport_Latitude', 'Airport_Longitude', 'ICAO_ID', 'Distance', 'Bearing', 'Distance_Kilometers', 'Bearing_Degrees'], axis=1)
null = pd.merge(null, nonstd, on='REMARKS', how='left')
null

,REMARKS,UAS_LOC,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION
0,Received a report via telephone of a UAS sight...,2 NM S of NCRCC,['rwy 1r'],[],"['IAD', 'CRC', 'IAD', 'TCT', 'IAD']",1/29/2019,4:35,2019-01-29T04:35Z,NaN,NCRCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),2 NM S of NCRCC,NaN
1,UAS:Fairfax County Police reported sighting a ...,1 NM E of NCRCC,['rwy 1r'],[],"['IAD', 'DCA', 'CRC', 'IAD', 'TCT']",2/4/2019,22:05,2019-02-04T22:05Z,NaN,NCRCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AIRPORT(?),1 NM E of NCRCC,NaN


In [23]:
null.to_csv('Non-std-2Remaining.csv', index = False)

In [64]:
# Find the matching records
new_df3 = new_df2[new_df2['NAME'].notnull()]
new_df3

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID
0,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT,FAT,0.50,N,0.926,0,Fresno Yosemite Intl,36.776556,-119.718833,KFAT
1,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR,BUR,0.50,S,0.926,180,Bob Hope,34.200694,-118.358667,KBUR
4,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA,LGA,3.00,SE,5.556,135,Laguardia,40.777250,-73.872611,KLGA
5,Reported observing a UAS operating over the 10...,1 NM S of EMT,EMT,1.00,S,1.852,180,San Gabriel Valley,34.086009,-118.034845,KEMT
6,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF,SMF,0.25,N,0.463,0,Sacramento Intl,38.695444,-121.590778,KSMF
7,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA,DCA,5.00,S,9.260,180,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA


In [65]:
# Save the matching records 
new_df3.to_csv('6Airports.csv', index = False)
new_df3 = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/6Airports.csv')

In [66]:
# Find the UAS lat/long coordinates using geopy
uas_lat = []
uas_long = []

for i in range(len(new_df3)):
    lat_airport = pd.to_numeric(new_df3['Airport_Latitude'][i])
    long_airport = pd.to_numeric(new_df3['Airport_Longitude'][i])
    b = pd.to_numeric(new_df3['Bearing_Degrees'][i])
    d = pd.to_numeric(new_df3['Distance_Kilometers'][i])
    
    origin = geopy.Point(lat_airport, long_airport)
    destination = geodesic(kilometers=d).destination(origin,b)
    
    lat2, lon2, = destination.latitude, destination.longitude
    
    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append new columns to UAS location dataframe
new_df3['UAS_Latitude'] = uas_lat
new_df3['UAS_Longitude'] = uas_long
new_df3

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,Distance_Kilometers,Bearing_Degrees,NAME,Airport_Latitude,Airport_Longitude,ICAO_ID,UAS_Latitude,UAS_Longitude
0,ATCT received report of a UAS flying .5 NM N o...,.5 NM N of FAT,FAT,0.50,N,0.926,0,Fresno Yosemite Intl,36.776556,-119.718833,KFAT,36.784900,-119.718833
1,N9042Q reported a UAS while on right base for ...,0.5 NM S of BUR,BUR,0.50,S,0.926,180,Bob Hope,34.200694,-118.358667,KBUR,34.192347,-118.358667
2,Reported black and white UAS at 1200 feet whil...,3 nm SE LGA,LGA,3.00,SE,5.556,135,Laguardia,40.777250,-73.872611,KLGA,40.741863,-73.826097
3,Reported observing a UAS operating over the 10...,1 NM S of EMT,EMT,1.00,S,1.852,180,San Gabriel Valley,34.086009,-118.034845,KEMT,34.069313,-118.034845
4,Sacramento Airport Authority notified of a mod...,.25 NM N of SMF,SMF,0.25,N,0.463,0,Sacramento Intl,38.695444,-121.590778,KSMF,38.699615,-121.590778
5,DCA-2: Aircraft landing DCA runway 1 observed ...,5 NM S of DCA,DCA,5.00,S,9.260,180,Ronald Reagan Washington Ntl,38.851444,-77.037722,KDCA,38.768030,-77.037722


In [67]:
# Export dataframe to csv file
new_df3 = new_df3.drop(['Distance','Bearing','Distance_Kilometers','Bearing_Degrees','ICAO_ID'], 1)
new_df3 = new_df3.rename({'Airport_Longitude': 'Origin_Longitude', 'Airport_Latitude': 'Origin_Latitude'}, axis=1)
new_df3 = pd.merge(new_df3, nonstd_air, left_on=['REMARKS', 'UAS_LOC'], right_on=['REMARKS','UASLOCATION'], how='left')
new_df3.to_csv('COMPLETED-Non-std-6Airports.csv', index = False)

In [ ]:
###########################################################################################################

In [84]:
# Find the records that reference runways
nonstd_rwy = nonstd[(nonstd['DATASET'] == 'RUNWAY')]
nonstd_rwy

,REMARKS,RUNWAY,DIRECTION,AIRPORT,dateonly,timeonly,DATE,CALLSIGN,POD,PRIMARYCODE,...,ACALTITUDE.CEDAR,ACHEADING.CEDAR,RELATIVECLOCKPOSITION.CEDAR,UASLONG.CEDAR,UASLAT.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,UASLOCATION,RWYLOCATION
0,Reported a UAS .25 NM N of their position whil...,['rwy27 '],"['25 NM N', '1 NM ']",[],1/30/2018,10:11,2018-01-30T10:11Z,NKS787,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM N of SAN,RWY 27
1,Aircraftreported a UAS 500 feet off their righ...,['rwy27r'],['18 NM NE'],"['27R', 'ORD']",2/19/2018,4:24,2018-02-19T04:24Z,ENY3709,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,18 NM NE ORD,RWY 27R
3,VFR aircraft reported observing a UAS pass wit...,['rwy23 '],[],[],4/21/2018,2:33,2018-04-21T02:33Z,N7425T,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM S SW1,RWY 23
4,Reported responding to two TCAS RA's over an o...,['rwy09'],[],['FNT'],6/21/2018,7:20,2018-06-21T07:20Z,IRO7552,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,6 NM W of FNT,RWY 09
5,Reported a UAS on a 6 NM final to RWY 22l. UAS...,['rwy22l'],"['6 NM ', '1 NM N']",[],7/3/2018,20:21,2018-07-03T20:21Z,SWA1156,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of BOS,RWY 22
6,PIC reported a white UAS off right side at sam...,['rwy17r'],['1 NM NNW'],['00F'],7/26/2018,19:21,2018-07-26T19:21Z,UPS2805,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM NNW SDF,RWY 17R
7,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,['rwy20 '],['25 NM E'],['50F'],8/16/2018,3:44,2018-08-16T03:44Z,N93EC,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,0.25 NM E MSY,RWY 20
8,"Reported observing a white UAS at 2,500 feet, ...",['rwy13 '],[],['LGA'],8/20/2018,5:11,2018-08-20T05:11Z,SKW3610,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,2 NM S of N90,RWY 13
9,Observed a black UAS with flashing red lights ...,['rwy25l'],['5 NM SE'],[],8/28/2018,2:22,2018-08-28T02:22Z,MSQT384,DEN,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1.5 NM SE of P50,RWY 25L
10,DCA-1: Two aircraft reported a drone 1 NM N of...,['rwy19c'],[],"['DCA', 'IAD', 'IAD', 'MOR']",9/7/2018,1:08,2018-09-07T01:08Z,UAL1192,NCRCC,UAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of IAD,RWY 19C


In [85]:
# Extract IDENTs from UAS location column
uas_loc = nonstd_rwy['UASLOCATION'].tolist()
uas_rwy = []

for i in range(len(uas_loc)):
    rwy = uas_loc[i].split(' ')[-1]
    
    if len(rwy) <= 4:
        uas_rwy.append(rwy)
    else:
        trim_air = rwy[-3:]
        uas_rwy.append(trim_air)
        
uas_rwy_df = pd.DataFrame()
uas_rwy_df['IDENT'] = uas_rwy
uas_rwy_df

,IDENT
0,SAN
1,ORD
2,SW1
3,FNT
4,BOS
5,SDF
6,MSY
7,N90
8,P50
9,IAD


In [86]:
# Add needed columns to the IDENTs dataframe
add_col = nonstd_rwy['REMARKS'].tolist()
uas_rwy_df['REMARKS'] = add_col
add_col1 = nonstd_rwy['UASLOCATION'].tolist()
uas_rwy_df['UASLOCATION'] = add_col1
add_col1 = nonstd_rwy['RWYLOCATION'].tolist()
uas_rwy_df['RWYLOCATION'] = add_col1
uas_rwy_df

,IDENT,REMARKS,UASLOCATION,RWYLOCATION
0,SAN,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,RWY 27
1,ORD,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,RWY 27R
2,SW1,VFR aircraft reported observing a UAS pass wit...,.25 NM S SW1,RWY 23
3,FNT,Reported responding to two TCAS RA's over an o...,6 NM W of FNT,RWY 09
4,BOS,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,RWY 22
5,SDF,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,RWY 17R
6,MSY,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,RWY 20
7,N90,"Reported observing a white UAS at 2,500 feet, ...",2 NM S of N90,RWY 13
8,P50,Observed a black UAS with flashing red lights ...,1.5 NM SE of P50,RWY 25L
9,IAD,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,RWY 19C


In [87]:
# Creating dictionary for all bearing abbreviations and their degrees
bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
}

# Separate Distances and bearings into two different columns
uas_loc = uas_rwy_df['UASLOCATION']
distances = []
bearings = []

for i in range(len(uas_loc)):
    split = uas_loc[i].split(' ')[0:3]
    
    if 'NM' in split[0]: 
        distance_nm = split[0][0:len(split[0]) - 2]
        bearing_abrev = split[1].replace('of', '')
        
    else: 
        distance_nm = split[0]
        bearing_abrev = split[2].replace('of', '')
    
    distances.append(distance_nm[0:])
    bearings.append(bearing_abrev)

# Create new dataframe for UAS locations
new_df = pd.DataFrame(uas_rwy_df['REMARKS'])
new_df['UAS_LOC'] = uas_rwy_df['UASLOCATION']
new_df['IDENT'] = uas_rwy_df['IDENT']
new_df['Distance'] = distances
new_df['Bearing'] = bearings
new_df['RWYLOCATION'] = uas_rwy_df['RWYLOCATION']

new_df

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,.25,N,RWY 27
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R
2,VFR aircraft reported observing a UAS pass wit...,.25 NM S SW1,SW1,.25,S,RWY 23
3,Reported responding to two TCAS RA's over an o...,6 NM W of FNT,FNT,6,W,RWY 09
4,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1,N,RWY 22
5,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,1,NNW,RWY 17R
6,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,0.25,E,RWY 20
7,"Reported observing a white UAS at 2,500 feet, ...",2 NM S of N90,N90,2,S,RWY 13
8,Observed a black UAS with flashing red lights ...,1.5 NM SE of P50,P50,1.5,SE,RWY 25L
9,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,1,N,RWY 19C


In [88]:
# Read in the linked airports and runways dataset
airport_runway = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/Airports&Runways.csv')
airport_runway

,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,IDENT,ICAO_ID,NAME,LATITUDE,LONGITUDE
0,5/23,-176.645456,51.882484,ADK,PADK,Adak,51.883583,-176.642482
1,4/22,-154.181625,56.938950,AKK,PAKH,Akhiok,56.938691,-154.182556
2,1/19,-161.492556,60.914625,Z13,PFZK,Akiachak,60.913810,-161.493329
3,E/W,-0.199980,-0.000081,KKI,NaN,Akiachak,60.907865,-161.435077
4,NW/SE,-0.199980,-0.000081,KKI,NaN,Akiachak,60.907865,-161.435077
...,...,...,...,...,...,...,...,...
24031,176/356,-99.265570,34.676719,LTS,KLTS,Altus AFB,34.667975,-99.267750
24032,18L/36R,-99.259598,34.667652,LTS,KLTS,Altus AFB,34.667975,-99.267750
24033,18R/36L,-99.273881,34.660327,LTS,KLTS,Altus AFB,34.667975,-99.267750
24034,1/19,-84.946625,36.350671,2A1,NaN,Jamestown Muni,36.349694,-84.946639


In [89]:
# Merge the dataset with the runway locations dataframe
uas_rwy_loc= pd.merge(new_df, airport_runway, on='IDENT', how='left')
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,.25,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04L/22R,-87.903438,41.991219,KORD,Chicago O'hare Intl,41.976629,-87.907253
2,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04R/22L,-87.887661,41.963319,KORD,Chicago O'hare Intl,41.976629,-87.907253
3,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253
4,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09R/27L,-87.900777,41.983948,KORD,Chicago O'hare Intl,41.976629,-87.907253
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,17R/35L,-81.295664,28.419116,KMCO,Orlando Intl,28.429389,-81.309000
79,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18L/36R,-81.322036,28.428531,KMCO,Orlando Intl,28.429389,-81.309000
80,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18R/36L,-81.326702,28.428491,KMCO,Orlando Intl,28.429389,-81.309000
81,"1647 EDT / 2047 UTC 8/21/2021\r\nRA13U, P28A, ...",1 NM W of PNS,PNS,1,W,RWY 26,8/26,-87.182890,30.473687,KPNS,Pensacola Intl,30.473417,-87.186611


In [90]:
# Split the designator for further use
designator = []

for i in range(len(uas_rwy_loc)):
    split = uas_rwy_loc['RWYLOCATION'][i].split(' ')
    designator.append(split[1])
    
uas_rwy_loc['RWY_DESIGNATOR'] = designator
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,.25,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04L/22R,-87.903438,41.991219,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R
2,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04R/22L,-87.887661,41.963319,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R
3,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R
4,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09R/27L,-87.900777,41.983948,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,17R/35L,-81.295664,28.419116,KMCO,Orlando Intl,28.429389,-81.309000,17L
79,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18L/36R,-81.322036,28.428531,KMCO,Orlando Intl,28.429389,-81.309000,17L
80,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18R/36L,-81.326702,28.428491,KMCO,Orlando Intl,28.429389,-81.309000,17L
81,"1647 EDT / 2047 UTC 8/21/2021\r\nRA13U, P28A, ...",1 NM W of PNS,PNS,1,W,RWY 26,8/26,-87.182890,30.473687,KPNS,Pensacola Intl,30.473417,-87.186611,26


In [91]:
# Add a new column to verify if the runway belongs to the airport (TRUE) or not (FALSE)
uas_rwy_loc['True/False'] = uas_rwy_loc.apply(lambda i: str(i.RWY_DESIGNATOR) in str(i.DESIGNATOR), axis=1)
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,.25,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27,True
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04L/22R,-87.903438,41.991219,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,False
2,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,04R/22L,-87.887661,41.963319,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,False
3,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,True
4,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09R/27L,-87.900777,41.983948,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,17R/35L,-81.295664,28.419116,KMCO,Orlando Intl,28.429389,-81.309000,17L,False
79,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18L/36R,-81.322036,28.428531,KMCO,Orlando Intl,28.429389,-81.309000,17L,False
80,1213 EDT / 1613 UTC 8/7/2021\r\nOrange County ...,0.125 NM E of MCO,MCO,0.125,E,RWY 17L,18R/36L,-81.326702,28.428491,KMCO,Orlando Intl,28.429389,-81.309000,17L,False
81,"1647 EDT / 2047 UTC 8/21/2021\r\nRA13U, P28A, ...",1 NM W of PNS,PNS,1,W,RWY 26,8/26,-87.182890,30.473687,KPNS,Pensacola Intl,30.473417,-87.186611,26,True


In [92]:
# Find all the records where the runways belong to the corresponding airport
uas_rwy_loc = uas_rwy_loc[uas_rwy_loc['True/False'] == True]
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,.25,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27,True
3,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,True
14,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1,N,RWY 22,04L/22R,-71.010357,42.366107,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True
15,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1,N,RWY 22,04R/22L,-71.004345,42.366586,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True
22,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,1,NNW,RWY 17R,17R/35L,-85.742481,38.168054,KSDF,Louisville Muhammad Ali Intl,38.174086,-85.736499,17R,True
23,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,0.25,E,RWY 20,2/20,-90.247923,29.995649,KMSY,Louis Armstrong New Orleans Intl,29.993278,-90.259028,20,True
27,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,1,N,RWY 19C,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,19C,True
33,Pilot reported a small brown UAS near the appr...,2 NM E of IWA,IWA,2,E,RWY 30R,12L/30R,-111.652837,33.310265,KIWA,Phoenix-Mesa Gateway,33.307824,-111.655459,30R,True
36,The Chief Flight Instructor for Embry-Riddle r...,0.25 NM E of DAB,DAB,0.25,E,RWY 07R,07R/25L,-81.051380,29.178229,KDAB,Daytona Beach Intl,29.179909,-81.058042,07R,True
44,PIC reported a white and redquadcopter at 5000...,2 NM E of ELP,ELP,2,E,RWY 26L,08R/26L,-106.377063,31.801433,KELP,El Paso Intl,31.807333,-106.376361,26L,True


In [93]:
# Export all records to csv file
uas_rwy_loc.to_csv('RWY22.csv', index = False)
uas_rwy_loc = pd.read_csv('C:/Users/Olga/Desktop/GMU/DAEN690/LEGO/RWY22.csv')

In [94]:
# Replace designator with respective degrees
designator = uas_rwy_loc['RWY_DESIGNATOR']
bearing = []

for i in range(len(uas_rwy_loc)):
    bearing.append(int(re.sub("\D","",designator[i]))*10)
    
uas_rwy_loc['RWY_BEARING'] = bearing
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,0.250,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27,True,270
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18.000,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,True,270
2,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04L/22R,-71.010357,42.366107,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220
3,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04R/22L,-71.004345,42.366586,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220
4,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,1.000,NNW,RWY 17R,17R/35L,-85.742481,38.168054,KSDF,Louisville Muhammad Ali Intl,38.174086,-85.736499,17R,True,170
5,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,0.250,E,RWY 20,2/20,-90.247923,29.995649,KMSY,Louis Armstrong New Orleans Intl,29.993278,-90.259028,20,True,200
6,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,1.000,N,RWY 19C,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,19C,True,190
7,Pilot reported a small brown UAS near the appr...,2 NM E of IWA,IWA,2.000,E,RWY 30R,12L/30R,-111.652837,33.310265,KIWA,Phoenix-Mesa Gateway,33.307824,-111.655459,30R,True,300
8,The Chief Flight Instructor for Embry-Riddle r...,0.25 NM E of DAB,DAB,0.250,E,RWY 07R,07R/25L,-81.051380,29.178229,KDAB,Daytona Beach Intl,29.179909,-81.058042,07R,True,70
9,PIC reported a white and redquadcopter at 5000...,2 NM E of ELP,ELP,2.000,E,RWY 26L,08R/26L,-106.377063,31.801433,KELP,El Paso Intl,31.807333,-106.376361,26L,True,260


In [95]:
# Converting distances to kilometers and append it to dataframe
uas_rwy_loc['Distance'] = pd.to_numeric(uas_rwy_loc['Distance'],errors = 'coerce')
dist_kilo = []

for i in range(len(uas_rwy_loc)):
    distanceKilo = uas_rwy_loc['Distance'][i] * 1.852 
    dist_kilo.append(distanceKilo)
    
uas_rwy_loc['Distance_Kilometers'] = dist_kilo
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING,Distance_Kilometers
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,0.250,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27,True,270,0.4630
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18.000,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,True,270,33.3360
2,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04L/22R,-71.010357,42.366107,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220,1.8520
3,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04R/22L,-71.004345,42.366586,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220,1.8520
4,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,1.000,NNW,RWY 17R,17R/35L,-85.742481,38.168054,KSDF,Louisville Muhammad Ali Intl,38.174086,-85.736499,17R,True,170,1.8520
5,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,0.250,E,RWY 20,2/20,-90.247923,29.995649,KMSY,Louis Armstrong New Orleans Intl,29.993278,-90.259028,20,True,200,0.4630
6,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,1.000,N,RWY 19C,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,19C,True,190,1.8520
7,Pilot reported a small brown UAS near the appr...,2 NM E of IWA,IWA,2.000,E,RWY 30R,12L/30R,-111.652837,33.310265,KIWA,Phoenix-Mesa Gateway,33.307824,-111.655459,30R,True,300,3.7040
8,The Chief Flight Instructor for Embry-Riddle r...,0.25 NM E of DAB,DAB,0.250,E,RWY 07R,07R/25L,-81.051380,29.178229,KDAB,Daytona Beach Intl,29.179909,-81.058042,07R,True,70,0.4630
9,PIC reported a white and redquadcopter at 5000...,2 NM E of ELP,ELP,2.000,E,RWY 26L,08R/26L,-106.377063,31.801433,KELP,El Paso Intl,31.807333,-106.376361,26L,True,260,3.7040


In [96]:
# Calculate UAS lat/long coordinates using geopy
uas_lat = []
uas_long = []


for i in range(len(uas_rwy_loc)):
    lat_rwy = pd.to_numeric(uas_rwy_loc['RUNWAY.LATITUDE'][i])
    long_rwy = pd.to_numeric(uas_rwy_loc['RUNWAY.LONGITUDE'][i])
    b = pd.to_numeric(uas_rwy_loc['RWY_BEARING'][i])
    d = pd.to_numeric(uas_rwy_loc['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame
uas_rwy_loc['UAS_Latitude'] = uas_lat
uas_rwy_loc['UAS_Longitude'] = uas_long
uas_rwy_loc

,REMARKS,UAS_LOC,IDENT,Distance,Bearing,RWYLOCATION,DESIGNATOR,RUNWAY.LONGITUDE,RUNWAY.LATITUDE,ICAO_ID,NAME,LATITUDE,LONGITUDE,RWY_DESIGNATOR,True/False,RWY_BEARING,Distance_Kilometers,UAS_Latitude,UAS_Longitude
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,0.250,N,RWY 27,9/27,-117.192633,32.734226,KSAN,San Diego Intl,32.733556,-117.189667,27,True,270,0.4630,32.734226,-117.197572
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,18.000,NE,RWY 27R,09L/27R,-87.910126,42.002880,KORD,Chicago O'hare Intl,41.976629,-87.907253,27R,True,270,33.3360,42.002175,-88.312503
2,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04L/22R,-71.010357,42.366107,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220,1.8520,42.353334,-71.024806
3,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,1.000,N,RWY 22,04R/22L,-71.004345,42.366586,KBOS,General Edward Lawrence Logan Intl,42.362944,-71.006389,22,True,220,1.8520,42.353813,-71.018794
4,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,1.000,NNW,RWY 17R,17R/35L,-85.742481,38.168054,KSDF,Louisville Muhammad Ali Intl,38.174086,-85.736499,17R,True,170,1.8520,38.151623,-85.738812
5,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,0.250,E,RWY 20,2/20,-90.247923,29.995649,KMSY,Louis Armstrong New Orleans Intl,29.993278,-90.259028,20,True,200,0.4630,29.991724,-90.249564
6,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,1.000,N,RWY 19C,01C/19C,-77.459557,38.958019,KIAD,Washington Dulles Intl,38.947444,-77.459944,19C,True,190,1.8520,38.941590,-77.463266
7,Pilot reported a small brown UAS near the appr...,2 NM E of IWA,IWA,2.000,E,RWY 30R,12L/30R,-111.652837,33.310265,KIWA,Phoenix-Mesa Gateway,33.307824,-111.655459,30R,True,300,3.7040,33.326959,-111.687289
8,The Chief Flight Instructor for Embry-Riddle r...,0.25 NM E of DAB,DAB,0.250,E,RWY 07R,07R/25L,-81.051380,29.178229,KDAB,Daytona Beach Intl,29.179909,-81.058042,07R,True,70,0.4630,29.179658,-81.046907
9,PIC reported a white and redquadcopter at 5000...,2 NM E of ELP,ELP,2.000,E,RWY 26L,08R/26L,-106.377063,31.801433,KELP,El Paso Intl,31.807333,-106.376361,26L,True,260,3.7040,31.795627,-106.415581


In [98]:
# Export dataframe to csv file
uas_rwy_loc = uas_rwy_loc.drop(['Distance','Bearing','Distance_Kilometers','RWY_BEARING','ICAO_ID','RWYLOCATION', 'DESIGNATOR','RWY_DESIGNATOR','True/False','LATITUDE','LONGITUDE'], 1)
uas_rwy_loc = uas_rwy_loc.rename({'RUNWAY.LONGITUDE': 'Origin_Longitude', 'RUNWAY.LATITUDE': 'Origin_Latitude'}, axis=1)
uas_rwy_loc = pd.merge(uas_rwy_loc, nonstd_rwy, on='REMARKS', how='left')
uas_rwy_loc.to_csv('COMPLETED-Non-std-22Runways.csv', index = False)

In [ ]:
########################################################################################################################

In [103]:
# Merge dataframes to find unmatched records
merge = pd.merge(uas_rwy_loc, nonstd_rwy, on=['REMARKS','RWYLOCATION'], how='outer', indicator=True)
merge

,REMARKS,UAS_LOC,IDENT,Origin_Longitude,Origin_Latitude,NAME,UAS_Latitude,UAS_Longitude,RUNWAY_x,DIRECTION_x,...,ACALTITUDE.CEDAR_y,ACHEADING.CEDAR_y,RELATIVECLOCKPOSITION.CEDAR_y,UASLONG.CEDAR_y,UASLAT.CEDAR_y,SUMMARY.CEDAR_y,QAFINDINGS.CEDAR_y,DATASET_y,UASLOCATION_y,_merge
0,Reported a UAS .25 NM N of their position whil...,.25 NM N of SAN,SAN,-117.192633,32.734226,San Diego Intl,32.734226,-117.197572,['rwy27 '],"['25 NM N', '1 NM ']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM N of SAN,both
1,Aircraftreported a UAS 500 feet off their righ...,18 NM NE ORD,ORD,-87.910126,42.002880,Chicago O'hare Intl,42.002175,-88.312503,['rwy27r'],['18 NM NE'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,18 NM NE ORD,both
2,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,-71.010357,42.366107,General Edward Lawrence Logan Intl,42.353334,-71.024806,['rwy22l'],"['6 NM ', '1 NM N']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of BOS,both
3,Reported a UAS on a 6 NM final to RWY 22l. UAS...,1 NM N of BOS,BOS,-71.004345,42.366586,General Edward Lawrence Logan Intl,42.353813,-71.018794,['rwy22l'],"['6 NM ', '1 NM N']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of BOS,both
4,PIC reported a white UAS off right side at sam...,1 NM NNW SDF,SDF,-85.742481,38.168054,Louisville Muhammad Ali Intl,38.151623,-85.738812,['rwy17r'],['1 NM NNW'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM NNW SDF,both
5,PIC reported a quadcopter UAS 0.25 NM E RWY 20...,0.25 NM E MSY,MSY,-90.247923,29.995649,Louis Armstrong New Orleans Intl,29.991724,-90.249564,['rwy20 '],['25 NM E'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,0.25 NM E MSY,both
6,DCA-1: Two aircraft reported a drone 1 NM N of...,1 NM N of IAD,IAD,-77.459557,38.958019,Washington Dulles Intl,38.941590,-77.463266,['rwy19c'],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1 NM N of IAD,both
7,Pilot reported a small brown UAS near the appr...,2 NM E of IWA,IWA,-111.652837,33.310265,Phoenix-Mesa Gateway,33.326959,-111.687289,['rwy30r'],['2 NM E'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,2 NM E of IWA,both
8,The Chief Flight Instructor for Embry-Riddle r...,0.25 NM E of DAB,DAB,-81.051380,29.178229,Daytona Beach Intl,29.179658,-81.046907,['rwy07r'],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,0.25 NM E of DAB,both
9,PIC reported a white and redquadcopter at 5000...,2 NM E of ELP,ELP,-106.377063,31.801433,El Paso Intl,31.795627,-106.415581,['rwy26l'],"['2 NM E', '2 NM ']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,2 NM E of ELP,both


In [101]:
# Save these records and send them to Lex
non_match = merge[merge['_merge'] == 'right_only']
non_match

,REMARKS,UAS_LOC,IDENT,Origin_Longitude,Origin_Latitude,NAME,UAS_Latitude,UAS_Longitude,RUNWAY_x,DIRECTION_x,...,ACALTITUDE.CEDAR_y,ACHEADING.CEDAR_y,RELATIVECLOCKPOSITION.CEDAR_y,UASLONG.CEDAR_y,UASLAT.CEDAR_y,SUMMARY.CEDAR_y,QAFINDINGS.CEDAR_y,DATASET_y,UASLOCATION_y,_merge
22,VFR aircraft reported observing a UAS pass wit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM S SW1,right_only
23,Reported responding to two TCAS RA's over an o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,6 NM W of FNT,right_only
24,"Reported observing a white UAS at 2,500 feet, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,2 NM S of N90,right_only
25,Observed a black UAS with flashing red lights ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,1.5 NM SE of P50,right_only
26,Pilot reported a black UAS orbiting while ENE ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,4 NM SW ATO,right_only
27,Reported observing a silver and white 4 rotor ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,4 NM W of A80,right_only
28,PIC reported a quad UAS at same altitude while...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,0.25 NM SE of PHX,right_only
29,"Reported a green quad copter 11.3 NM W at 2,90...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,11.3 NM W MIA,right_only
30,Reported a model airplane operating .25 NM E o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RUNWAY,.25 NM E of ZKC,right_only


In [46]:
non_match.to_csv('1stPass_All_Points_Non-std-479Runways -- 9NoMatch.csv', index = False)